In [4]:
# !pip install influxdb-client

^C


In [6]:
import os
os.environ["INFLUXDB_TOKEN"] = (
    "YOUR_INFLUX_TOKEN")

In [9]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "Ubru Farm"
url = "http://localhost:8086"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)


In [61]:
!pip install pytz


In [66]:
from datetime import datetime
from influxdb_client import InfluxDBClient
import pytz

# Initialize your InfluxDB client
client = InfluxDBClient(url="http://localhost:8086", token="Your access token", org="Ubru Farm")

query_api = client.query_api()

# Define your time range and window period
start_time = "2024-09-01T00:00:00Z"
stop_time = "2024-10-12T00:00:00Z"

# Replace "temperature" with your actual numeric field name
query = f"""
from(bucket: "Hydropolic-Farm")
  |> range(start: {start_time}, stop: {stop_time})
  |> filter(fn: (r) => r["_field"] == "Atmophere-Humidity" or r["_field"] == "Atmophere-Temperature" or r["_field"] == "Light" or r["_field"] == "Water-Temperature-Box-A" or r["_field"] == "Water-Temperature-Box-B")
  |> yield(name: "mean")
"""
def format_time(dt):
    # Convert the datetime object to Thailand time zone
    thailand_tz = pytz.timezone('Asia/Bangkok')
    local_time = dt.astimezone(thailand_tz)
    return local_time.strftime('%Y-%m-%d %H:%M:%S %Z')

try:
    tables = query_api.query(query, org="Ubru Farm")

    for table in tables:
        print(f"Table ({table}):")
        for record in table.records:
            # Convert the _time field to a readable format
            readable_time = format_time(record['_time'] )
            print(f"Time: {readable_time}, Field: {record['_field']}, Value: {record['_value']}")
        print("\n")
except Exception as e:
    print(f"An error occurred: {e}")

Table (FluxTable() columns: 8, records: 8):
Time: 2024-09-09 15:11:15 +07, Field: Atmophere-Humidity, Value: 45.0
Time: 2024-09-09 15:14:20 +07, Field: Atmophere-Humidity, Value: 45.0
Time: 2024-09-09 15:20:11 +07, Field: Atmophere-Humidity, Value: 45.0
Time: 2024-09-09 15:20:30 +07, Field: Atmophere-Humidity, Value: 45.0
Time: 2024-09-12 15:06:37 +07, Field: Atmophere-Humidity, Value: 45.0
Time: 2024-09-12 15:12:37 +07, Field: Atmophere-Humidity, Value: 85.0
Time: 2024-09-12 15:29:44 +07, Field: Atmophere-Humidity, Value: 12.0
Time: 2024-09-12 16:14:43 +07, Field: Atmophere-Humidity, Value: 90.0


Table (FluxTable() columns: 8, records: 8):
Time: 2024-09-09 15:11:15 +07, Field: Atmophere-Temperature, Value: 26.0
Time: 2024-09-09 15:14:20 +07, Field: Atmophere-Temperature, Value: 26.0
Time: 2024-09-09 15:20:11 +07, Field: Atmophere-Temperature, Value: 26.0
Time: 2024-09-09 15:20:30 +07, Field: Atmophere-Temperature, Value: 26.0
Time: 2024-09-12 15:06:37 +07, Field: Atmophere-Temperatur

In [71]:
from datetime import datetime
from influxdb_client import InfluxDBClient
import pytz

# Initialize your InfluxDB client
client = InfluxDBClient(url="http://localhost:8086", token="", org="")

query_api = client.query_api()

# Define your time range and window period
start_time = "2024-09-01T00:00:00Z"
stop_time = "2024-10-12T00:00:00Z"

# Replace "temperature" with your actual numeric field name
query = f"""
from(bucket: "Hydropolic-Farm")
  |> range(start: {start_time}, stop: {stop_time})
  |> filter(fn: (r) => r["_field"] == "Atmophere-Humidity" or r["_field"] == "Atmophere-Temperature" or r["_field"] == "Light" or r["_field"] == "Water-Temperature-Box-A" or r["_field"] == "Water-Temperature-Box-B")
  |> yield(name: "mean")
"""

def format_time(dt):
    # Convert the datetime object to Thailand time zone
    thailand_tz = pytz.timezone('Asia/Bangkok')
    local_time = dt.astimezone(thailand_tz)
    return local_time.strftime('%Y-%m-%d %H:%M:%S %Z')

try:
    tables = query_api.query(query, org="")
    
    # Collect all records
    records_by_date = {}
    
    for table in tables:
        for record in table.records:
            timestamp = format_time(record['_time'])
            if timestamp not in records_by_date:
                records_by_date[timestamp] = []
            records_by_date[timestamp].append(f"{record['_field']}, Value: {record['_value']}")
    
    # Sort dates in descending order
    sorted_dates = sorted(records_by_date.keys(), reverse=True)
    
    # Print records grouped by date
    # [:5] use to cut the latest 5 record only
    for date in sorted_dates[:5]:   
        records = " ; ".join(records_by_date[date])
        print(f"{date} ; {records}")

except Exception as e:
    print(f"An error occurred: {e}")


2024-09-12 16:14:43 +07 ; Atmophere-Humidity, Value: 90.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-12 15:29:44 +07 ; Atmophere-Humidity, Value: 12.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-12 15:12:37 +07 ; Atmophere-Humidity, Value: 85.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-12 15:06:37 +07 ; Atmophere-Humidity, Value: 45.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-09 15:20:30 +07 ; Atmophere-Humidity, Value: 45.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0


# Using library to fetch realtime data with 10 days past

In [1]:
from datetime import datetime, timedelta
from influxdb_client import InfluxDBClient
import pytz

# Initialize your InfluxDB client
client = InfluxDBClient(url="http://localhost:8086", token="Your access token", org="Your influxdb org")

query_api = client.query_api()

# Calculate dynamic time range (last 10 days)
end_time = datetime.utcnow()  # current time in UTC
start_time = end_time - timedelta(days=10)  # 10 days ago

# Format times in ISO 8601 format (for InfluxDB query)
start_time_str = start_time.isoformat() + 'Z'
end_time_str = end_time.isoformat() + 'Z'

# Replace "temperature" with your actual numeric field name
query = f"""
from(bucket: "Hydropolic-Farm")
  |> range(start: {start_time_str}, stop: {end_time_str})
  |> filter(fn: (r) => r["_field"] == "Atmophere-Humidity" or r["_field"] == "Atmophere-Temperature" or r["_field"] == "Light" or r["_field"] == "Water-Temperature-Box-A" or r["_field"] == "Water-Temperature-Box-B")
  |> yield(name: "mean")
"""

def format_time(dt):
    # Convert the datetime object to Thailand time zone
    thailand_tz = pytz.timezone('Asia/Bangkok')
    local_time = dt.astimezone(thailand_tz)
    return local_time.strftime('%Y-%m-%d %H:%M:%S %Z')

try:
    tables = query_api.query(query, org="Your influxdb org")
    
    # Collect all records
    records_by_date = {}
    
    for table in tables:
        for record in table.records:
            timestamp = format_time(record['_time'])
            if timestamp not in records_by_date:
                records_by_date[timestamp] = []
            records_by_date[timestamp].append(f"{record['_field']}, Value: {record['_value']}")
    
    # Sort dates in descending order
    sorted_dates = sorted(records_by_date.keys(), reverse=True)
    
    # Print records grouped by date
    # [:5] use to cut the latest 5 records only
    for date in sorted_dates[:5]:   
        records = " ; ".join(records_by_date[date])
        print(f"{date} ; {records}")

except Exception as e:
    print(f"An error occurred: {e}")


C:\Users\admin\AppData\Local\Temp\ipykernel_4328\2654839037.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow()  # current time in UTC


2024-09-12 16:14:43 +07 ; Atmophere-Humidity, Value: 90.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-12 15:29:44 +07 ; Atmophere-Humidity, Value: 12.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-12 15:12:37 +07 ; Atmophere-Humidity, Value: 85.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-12 15:06:37 +07 ; Atmophere-Humidity, Value: 45.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
2024-09-09 15:20:30 +07 ; Atmophere-Humidity, Value: 45.0 ; Atmophere-Temperature, Value: 26.0 ; Light, Value: 25400.0 ; Water-Temperature-Box-A, Value: 20.0 ; Water-Temperature-Box-B, Value: 19.0
